# Sistema de Recomendación Basado en Contenido

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

Cargamos los datos limpios de las películas.

In [2]:
df = pd.read_csv('data/metadata_clean.csv')

df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995


Cargamos los datos originales de las películas y añadimos dos variables al conjunto de datos limpio.

In [3]:
orig_df = pd.read_csv('data/movies_metadata.csv', low_memory=False)

df['overview'], df['id'] = orig_df['overview'], orig_df['id']

df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


1. Importamos las librerías necesarias para vectorizar texto. 
2. Eliminamos las ***stopwords*** del inglés y sustituimos los NaN. 
3. Finalmente, aplicamos el algoritmo de vectorización de texto.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')

df['overview'] = df['overview'].fillna('')

tfidf_matrix = tfidf.fit_transform(df['overview'])

tfidf_matrix.shape

(45466, 75827)

1. Importamos el kernel lineal para calcular el producto escalar.
2. Calculamos la matriz de similitud del coseno.

In [5]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

Construir una correspondencia inversa de índices y títulos de películas, y eliminar los títulos duplicados, si los hay.

In [6]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

Desarrollamos la función que toma el título de la película como entrada y da recomendaciones 

Funcionamiento:
1. Obtener el índice de la película que coincide con el título
1. Obtener las puntuaciones de similitud pairwsie de todas las películas con esa película y convertirlo en una lista de tuplas como se ha descrito anteriormente
1. Ordenar las películas en función de las puntuaciones de similitud coseno.
1. Obtener las puntuaciones de las 10 películas más similares. Ignorar la primera película.
1. Obtener los índices de las películas
1. Devuelve las 10 películas más parecidas

In [7]:
def content_recommender(title, cosine_sim=cosine_sim, df=df, indices=indices):
    
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    movie_indices = [i[0] for i in sim_scores]

    return df['title'].iloc[movie_indices]

Obtenemos la recomendación.

In [8]:
content_recommender('Jumanji')

21633         Table No. 21
45253                 Quiz
41573         Snowed Under
35509             The Mend
44376    Liar Game: Reborn
17223       The Dark Angel
8801               Quintet
6166             Brainscan
30981         Turkey Shoot
9503             Word Wars
Name: title, dtype: object